In [3]:
import pandas as pd 
import sklearn  as skl
import matplotlib as mtp
import seaborn as sns
import statsmodels.tsa as smt
import scipy 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import statsmodels as sm
from openpyxl import load_workbook
from datetime import datetime, timedelta, time 
from collections import defaultdict
import os 
import numpy as np
from __future__ import annotations
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from pandas.api.types import is_number, is_integer_dtype, is_float_dtype


In [8]:
#######################################################################
############################ DATA CLEANING ############################
#######################################################################

# ---------- Sheet listing & loading ----------
def list_sheets(path: str, drop: List[str]) -> List[str]:
    """Return sheet names from an Excel file, excluding any in `drop`."""
    xls = pd.ExcelFile(path)
    keep = [s for s in xls.sheet_names if s not in set(drop)]
    return keep


def load_sheets_as_dict(path: str) -> Dict[str, pd.DataFrame]:
    """Read all sheets from an Excel file into a dict[name -> DataFrame]."""
    xls = pd.ExcelFile(path)
    return {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in xls.sheet_names}


# ---------- LDZ filtering & tagging ----------
def filter_ldz_and_tag(
    dct: Dict[str, pd.DataFrame],
    country_codes: List[str],
    ldz_regex: str = r'(?i)LDZ'
) -> Dict[str, pd.DataFrame]:
    """
    Keep only LDZ columns for sheets whose key contains 'gas' and suffix them with a country code,
    assigned in the order of occurrence. Non-gas sheets are kept as-is.
    """
    out: Dict[str, pd.DataFrame] = {}
    i = 0
    for k, df in dct.items():
        if 'gas' in k.lower():
            sub = df.filter(regex=ldz_regex)
            if not sub.empty:
                cn = country_codes[i] if i < len(country_codes) else f"CC{i+1}"
                i += 1
                sub = df.rename(columns=lambda c: f"{c}_{cn}" if 'LDZ' in c else c)
                sub = sub.iloc[::-1]  # reverse order if desired
                out[k] = sub
            else:
                out[k] = df.copy()
        else:
            out[k] = df.copy()
    return out


# ---------- Merge helpers ----------
def merge_core(
    full_df: pd.DataFrame,
    ldz_dict: Dict[str, pd.DataFrame],
    keys: Tuple[str, str] = ('Gas Day Start', 'Gas Day End'),
) -> pd.DataFrame:
    """
    Merge storage & gas on `keys` (outer), then join each LDZ frame by index (outer).
    """
    merged = full_df.copy()
    for k, small in ldz_dict.items():
        merged = merged.join(small, how='outer', rsuffix=f"_{k}")
    return merged


def filter_since(df: pd.DataFrame, date_col: str, cutoff: str) -> pd.DataFrame:
    """Keep rows where date_col > cutoff."""
    if date_col not in df.columns:
        return df
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    return df[df[date_col] > pd.Timestamp(cutoff)]


# ---------- Column selection ----------
def select_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """Keep only numeric columns."""
    return df.select_dtypes(include='number').copy()


def select_numeric_by_checker(df: pd.DataFrame) -> pd.DataFrame:
    """Alternative numeric column selector using dtype checkers."""
    keep_cols = [c for c in df.columns if is_integer_dtype(df[c]) or is_float_dtype(df[c])]
    return df[keep_cols].copy()


# ---------- Feature engineering ----------
def add_log_changes(
    df: pd.DataFrame,
    cols: List[str],
    periods: int = 1,
    suffix: str = '_change',
) -> pd.DataFrame:
    """
    Add log change columns: log(x_t / x_{t-1}) by default.
    """
    out = df.copy()
    for col in cols:
        if col not in out.columns:
            continue
        prev = out[col].shift(-periods)
        chg = np.log(out[col] / prev)
        out[f"{col}{suffix}"] = chg.replace([np.inf, -np.inf], np.nan)
    return out


# ---------- One-shot pipeline ----------
def run_pipeline(
    stephen_path: str,
    full_data_path: str,
    country_codes: List[str],
    ldz_regex: str = r'(?i)LDZ',
    cutoff_date: str = '2022-01-01',
    change_cols: Optional[List[str]] = None
) -> Dict[str, pd.DataFrame]:
    """
    Execute the whole flow and return a dict of useful DataFrames.
    Keys: ldz_dct, merged, merged_since, numeric_only, with_changes (if applicable)
    """
    # 1) Load Stephen workbook
    stephen_dict = load_sheets_as_dict(stephen_path)
    ldz_tagged = filter_ldz_and_tag(stephen_dict, country_codes, ldz_regex=ldz_regex)

    # 2) Load full data workbook
    full_data_df = pd.read_excel(full_data_path)

    # 3) Merge
    merged = merge_core(full_data_df, ldz_tagged)

    # 4) Filter by date
    merged_since = filter_since(merged, 'Gas Day End', cutoff_date)

    # 5) Numeric-only
    numeric_only = select_numeric(merged_since)

    # 6) Optional log changes
    if change_cols:
        with_changes = add_log_changes(merged_since, change_cols)
    else:
        with_changes = merged_since.copy()

    return {
        "merged_since": merged_since,
        "numeric_only": numeric_only,
        "with_changes": with_changes,
    }


In [9]:
stephen_data = r'C:\Users\Deram1J\OneDrive - EDF\wmo-trm-trm-dashboard-services\Joakim\Trading\GasData.xlsx'
full_data = r'C:\Users\Deram1J\OneDrive - EDF\wmo-trm-trm-dashboard-services\Joakim\Trading\FullData.xlsx'
country_codes = ['GER', 'FR', 'UK', 'DL', 'IT', 'BLG']


result = run_pipeline(
        stephen_path=stephen_data, 
        full_data_path=full_data, 
        country_codes=country_codes
    )


In [10]:
result['merged_since']

,Gas Day End,LNG Inventory (10³ ㎥ LNG),LNG Inventory (~GWh),Technical Capacity / DTMI (10³ ㎥ LNG),Technical Capacity / DTMI (~GWh),Send-Out (GWh/d),DTRS (GWh/d),Contracted Capacity (GWh/d),Available Capacity (GWh/d) LNG,Gas in storage (TWh),...,Volume,Unit,Loading Geography,Load Region,Load State,Load Berth,Load Zone,Load Date From,Discharge Geography,Discharge Region
0,2025-10-26,4796.55,31354.99,9487.06,62016.80,3889.4,8085.1,4611.1167,943.4956,946.1407,...,133825.0,m3 LNG,Algeria,"Europe, Middle East, Africa",NaN,Sonatrach GL37 - M7 (Arzew),"Mediterranean, North Africa",2024-09-26 07:00:03,Spain,"Europe, Middle East, Africa"
1,2025-10-25,5072.96,33162.97,9487.06,62018.84,4328.5,8022.8,4614.5399,877.7510,945.7589,...,140972.0,m3 LNG,Norway,"Europe, Middle East, Africa",NaN,Melkoya LNG Terminal and Liquification Plant (...,North West Europe,2024-09-24 16:41:41,Finland,"Europe, Middle East, Africa"
2,2025-10-24,4983.78,32503.00,9487.06,61872.30,4122.9,8028.7,4588.1449,910.0771,945.6449,...,145000.0,m3 LNG,Algeria,"Europe, Middle East, Africa",NaN,Arzew - El Djedid - LNG & LPG Export Terminal ...,"Mediterranean, North Africa",2024-09-28 13:08:13,France (Southern),"Europe, Middle East, Africa"
3,2025-10-23,4874.10,31743.62,9487.06,61786.51,4169.9,8028.6,4566.4149,931.7670,945.2333,...,146000.0,m3 LNG,Russian Federation,"Europe, Middle East, Africa",NaN,NaN,"Russia, FSU",2024-09-21 02:20:44,Netherlands,"Europe, Middle East, Africa"
4,2025-10-22,5204.28,34003.50,9487.06,61986.14,4057.8,8028.6,4483.9512,1014.1691,945.3606,...,140000.0,m3 LNG,Louisiana,Americas,Texas,Sabine Pass LNG Terminal,North America,2024-09-16 14:15:43,Netherlands,"Europe, Middle East, Africa"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2022-01-06,4804.23,32490.31,7443.67,50340.46,3237.3,5571.7,0.0000,0.0000,591.8799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1390,2022-01-05,4941.13,33387.68,7443.67,50297.58,3002.7,5571.7,0.0000,0.0000,596.1655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1391,2022-01-04,4063.07,27482.58,7443.67,50348.94,2916.2,5571.7,0.0000,0.0000,598.9044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1392,2022-01-03,4199.56,28381.32,7443.67,50305.55,2644.3,5571.7,0.0000,0.0000,600.4137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [26]:
storage_df = pd.read_excel(storage_data)
gas_df = pd.read_excel(gas_data)
gas_df.columns

Index(['Status', 'Gas Day Start (status at 6AM CET)', 'Gas Day End',
       'LNG Inventory (10³ ㎥ LNG)', 'LNG Inventory (~GWh)',
       'Technical Capacity / DTMI (10³ ㎥ LNG)',
       'Technical Capacity / DTMI (~GWh)', 'Send-Out (GWh/d)', 'DTRS (GWh/d)',
       'Contracted Capacity (GWh/d)', 'Available Capacity (GWh/d)'],
      dtype='object')